In [1]:
import pandas as pd  
from random import random
 
pdata = pd.read_csv('oct_march.csv',names=['chid','views','subcriber','videocount','date']) 


In [2]:
pdata=pdata.drop(pdata.index[0])

In [3]:
channel_groups=pdata.groupby('chid')
grps=[channel_groups.get_group(x) for x in channel_groups.groups]

In [4]:

def _load_data(data, n_prev = 4):  
    """
    data should be pd.DataFrame()
    """

    docX, docY = [], []
    for i in range(len(data)-n_prev):
        docX.append(data.iloc[i:i+n_prev-1].as_matrix())
        docY.append(data.iloc[i+n_prev].as_matrix())
    alsX = np.array(docX)
    alsY = np.array(docY)

    return alsX, alsY

def train_test_split(df, test_size=0):  
    """
    This just splits data to training and testing parts
    """
    ntrn = round(len(df) * (1 - test_size))

    X_train, y_train = _load_data(df.iloc[0:ntrn])

    return (X_train, y_train)

In [5]:
def m():
    in_out_neurons = 3  

    hidden_neurons = 300


    model = Sequential()  

    model.add(LSTM(batch_input_shape=(1,3,3),units=hidden_neurons,use_bias=True, return_sequences=True,stateful=True)) 
    model.add(Dropout(0.2))
    model.add(LSTM(units=150,stateful=True,use_bias=True))
    model.add(Dropout(0.2))
    model.add(Dense(units=10,use_bias=True))  
    model.add(Activation("tanh"))
    model.add(Dropout(0.2))
    model.add(Dense(units=3,use_bias=True))
    model.add(Activation("linear"))
    model.add(Dropout(0.2))
    model.compile(loss="mean_squared_error", optimizer="rmsprop") 
    return model

In [6]:
%matplotlib notebook
from keras.models import Sequential  
from keras.layers.core import Dense, Activation,Dropout 
from keras.layers.recurrent import LSTM

import numpy as np
l=np.ndarray((len(grps),3,2))
for i in range(len(grps)):
    p=grps[i]
    p=p.sort_values(by='date')
    del p['date']
    del p['chid']
    p['views']=p.views.astype(int)
    l[i][0][0]=p['views'].max()- p['views'].min()
    l[i][0][1]=np.asarray(p['views'], dtype=np.float).mean()
    p['views']= (p['views'] - l[i][0][1] )/ l[i][0][0]
    p['subcriber']=p['subcriber'].astype(int)
    l[i][1][0]=p['subcriber'].max() - p['subcriber'].min()
    l[i][1][1]=np.asarray(p['subcriber'], dtype=np.float).mean()
    p['subcriber']= (p['subcriber'] - np.asarray(p['subcriber'], dtype=np.float).mean()) / (p['subcriber'].max() - p['subcriber'].min())
    p['videocount']=p['videocount'].astype(int)
    l[i][2][0]=p['videocount'].max() - p['videocount'].min()
    l[i][2][1]=np.asarray(p['videocount'], dtype=np.float).mean()
    p['videocount']= (p['videocount'] - np.asarray(p['videocount'], dtype=np.float).mean()) / (p['videocount'].max() - p['videocount'].min())
    (X_train, y_train)= train_test_split(p)  # retrieve data
    X_train=X_train.reshape((-1,3,3))
    M=m() 
    M.fit(x=X_train, y=y_train,batch_size=1,epochs=3, verbose=2,shuffle=False) 
    M.reset_states()

Using TensorFlow backend.


Epoch 1/3
3s - loss: nan
Epoch 2/3
2s - loss: nan
Epoch 3/3
2s - loss: nan
Epoch 1/3
4s - loss: nan
Epoch 2/3
3s - loss: nan
Epoch 3/3
2s - loss: nan
Epoch 1/3
3s - loss: 0.0403
Epoch 2/3
2s - loss: 0.0300
Epoch 3/3
2s - loss: 0.0286
Epoch 1/3
3s - loss: 0.0274
Epoch 2/3
2s - loss: 0.0268
Epoch 3/3
2s - loss: 0.0205
Epoch 1/3
3s - loss: 0.0253
Epoch 2/3
3s - loss: 0.0300
Epoch 3/3
2s - loss: 0.0219
Epoch 1/3
3s - loss: 0.0531
Epoch 2/3
2s - loss: 0.0548
Epoch 3/3
2s - loss: 0.0495
Epoch 1/3
3s - loss: 0.0876
Epoch 2/3
2s - loss: 0.0591
Epoch 3/3
2s - loss: 0.0618
Epoch 1/3
4s - loss: 0.0657
Epoch 2/3
2s - loss: 0.0478
Epoch 3/3
2s - loss: 0.0393
Epoch 1/3
4s - loss: 0.0487
Epoch 2/3
2s - loss: 0.0443
Epoch 3/3
2s - loss: 0.0384
Epoch 1/3
4s - loss: 0.0360
Epoch 2/3
3s - loss: 0.0414
Epoch 3/3
2s - loss: 0.0398
Epoch 1/3
4s - loss: 0.0250
Epoch 2/3
2s - loss: 0.0268
Epoch 3/3
2s - loss: 0.0257
Epoch 1/3
4s - loss: nan
Epoch 2/3
2s - loss: nan
Epoch 3/3
3s - loss: nan
Epoch 1/3
4s - loss

In [7]:
import math
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2017, 4, 1)
end_date = date(2017, 7, 1)
p=np.empty((10*31,3,3))
predicted=np.ndarray((10*31,3))
validation=np.empty((10*31,3))
g=grps[0]
g['views']=g.views.astype(int)
g['subcriber']=g.subcriber.astype(int)
g['videocount']=g.videocount.astype(int)
X=g.sort_values(by='date')
print ("prediction for "+X['chid'].values[0])
del g['date']
del g['chid']
del X['date']
del X['chid']
M.reset_states()
X=X.head(3)
p[0]=X.values.reshape((1,3,3))
i=1
for k in range(3):
    validation[k]=(X.values)[k]
for single_date in daterange(date(2016,10,1), date(2017,3,31)):
    p[i][0]=p[i-1][1]
    p[i][1]=p[i-1][2]
    p[i][2]=M.predict(p[i-1].reshape((1,3,3)))
    validation[i+3]=p[i][2]
    i=i+1


/home/shubham/anaconda3/envs/tensorflowcpu/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/shubham/anaconda3/envs/tensorflowcpu/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/shubham/anaconda3/envs/tensorflowcpu/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

prediction for UC60GHVk_6ddgqhHwcg


In [8]:
from cmath import sqrt
sqr_err=0
n=np.ndarray(3)
for x in range(i):
    n[0]=((g.values)[x][0]-l[0][0][1])/l[0][0][0]
    n[1]=((g.values)[x][1]-l[0][1][1])/l[0][1][0]
    n[2]=((g.values)[x][2]-l[0][2][1])/l[0][2][0]
    sqr_err+=(n-validation[x])**2
mse=np.mean(sqr_err)
rmse=mse**0.5
print ("the rmse of predicted values is"+ str(rmse))
j=0
for single_date in daterange(start_date, end_date):
    p[i][0]=p[i-1][1]
    p[i][1]=p[i-1][2]
    p[i][2]=M.predict(p[i-1].reshape((1,3,3)))
    predicted[j][0]=p[i][2][0]*l[0][0][0]+l[0][0][1]
    predicted[j][1]=p[i][2][1]*l[0][1][0]+l[0][1][1]
    predicted[j][2]=p[i][2][2]*l[0][2][0]+l[0][2][1]
    print (single_date,predicted[j])
    i=i+1

/home/shubham/anaconda3/envs/tensorflowcpu/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


the rmse of predicted values is(nan+nanj)
2017-04-01 [  3.89076881e+06   2.80537843e+05   7.00000000e+00]
2017-04-02 [  3.89076881e+06   2.80537843e+05   7.00000000e+00]
2017-04-03 [  3.89076881e+06   2.80537843e+05   7.00000000e+00]
2017-04-04 [  3.89076881e+06   2.80537843e+05   7.00000000e+00]
2017-04-05 [  3.89076881e+06   2.80537843e+05   7.00000000e+00]
2017-04-06 [  3.89076881e+06   2.80537843e+05   7.00000000e+00]
2017-04-07 [  3.89076881e+06   2.80537843e+05   7.00000000e+00]
2017-04-08 [  3.89076881e+06   2.80537843e+05   7.00000000e+00]
2017-04-09 [  3.89076881e+06   2.80537843e+05   7.00000000e+00]
2017-04-10 [  3.89076881e+06   2.80537843e+05   7.00000000e+00]
2017-04-11 [  3.89076881e+06   2.80537843e+05   7.00000000e+00]
2017-04-12 [  3.89076881e+06   2.80537843e+05   7.00000000e+00]
2017-04-13 [  3.89076881e+06   2.80537843e+05   7.00000000e+00]
2017-04-14 [  3.89076881e+06   2.80537843e+05   7.00000000e+00]
2017-04-15 [  3.89076881e+06   2.80537843e+05   7.00000000e+00